In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math as math
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import Matern, RationalQuadratic,ExpSineSquared,PairwiseKernel
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
import joblib

In [2]:
columns = ["x","y","z","w","data","error"]
datatab = pd.read_table(r"C:\Users\seanw\OneDrive\Desktop\GitHub\FYP\Data\Updateddata.txt",names = columns)
datatab
x = datatab['x']
y = datatab['y']
z = datatab['z']
w = datatab['w']
data = datatab['data']
error = datatab['error']

# First Mass
datam1 = datatab[0:250]
xm1 = datam1['x'].values
ym1 = datam1['y'].values
zm1 = datam1['z'].values
wm1 = datam1['w'].values
dm1 = datam1['data'].values
em1 = datam1['error'].values
X1 = np.column_stack((xm1, ym1, zm1, wm1))

# Second Mass
datam2 = datatab[250:500]
xm2 = datam2['x'].values
ym2 = datam2['y'].values
zm2 = datam2['z'].values
wm2 = datam2['w'].values
dm2 = datam2['data'].values
em2 = datam2['error'].values
X2 = np.column_stack((xm2, ym2, zm2, wm2))

# Third Mass
datam3 = datatab[500:750]
xm3 = datam3['x'].values
ym3 = datam3['y'].values
zm3 = datam3['z'].values
wm3 = datam3['w'].values
dm3 = datam3['data'].values
em3 = datam3['error'].values
X3 = np.column_stack((xm3, ym3, zm3, wm3))

# Fourth Mass
datam4 = datatab[750:]
xm4 = datam4['x'].values
ym4 = datam4['y'].values
zm4 = datam4['z'].values
wm4 = datam4['w'].values
dm4 = datam4['data'].values
em4 = datam4['error'].values
X4 = np.column_stack((xm4, ym4, zm4, wm4))

X = np.column_stack((x,y, z, w))
D = np.concatenate((dm1,dm2,dm3,dm4))
E = np.concatenate((em1,em2,em3,em4))

### Using Cross Validation to shortlist the best models
- Divide data into 90-10 train-test split
- Use 10-fold cross validation on the train set to shortlist the best models
- I will shortlist the models based on 6 metrics which are RMSE, MAE, R², Adjusted R², Figure of Merit and Pearson Coefficient
- After shortlisting I will build the final model on the entire train set and evaluate it on the test set
- I will also build MCMC models using each of the model types found in the shortlisting step
- I will also compare the MCMC models with the point estimate models using the same metrics

In [4]:
indices = np.arange(X.shape[0])
X_trainval, X_test, D_trainval, D_test, E_trainval, E_test, indices_trainval, indices_test = train_test_split(X, D,E,indices, test_size=0.1, random_state=42)
folds = 10

In [5]:
def build_gpr(set_kernel, set_noise_type, E_train=None):
    """
    Builds a GaussianProcessRegressor given a kernel and noise configuration.
    
    Parameters:
    -----------
    set_kernel : kernel
        A scikit-learn kernel object, e.g. RBF(), Matern(), etc.
    set_noise_type  : str
        A code describing how we handle noise:
          - 'fixed_alpha': use alpha = E_train^2 (no WhiteKernel)
          - 'white_no_error': WhiteKernel with some default noise level
          - 'white_mean_error': WhiteKernel seeded by E_train.mean()
          - 'white_minmax_error': WhiteKernel with bounds from min/max
    E_train : ndarray or float, optional
        If you need the training errors for alpha or WhiteKernel initialization.
    Returns:
    --------
    gpr : GaussianProcessRegressor
        The constructed GPR model with the requested noise approach.
    """

    # Set defaults
    normalize_y = True
    n_restarts_optimizer = 20  
    random_state = 42

    if set_noise_type == 'fixed_alpha':
        if E_train is None:
            raise ValueError("E_train must be provided for 'fixed_alpha' noise config.")
        alpha_val = (E_train)**2
        return GaussianProcessRegressor(kernel=set_kernel, alpha=alpha_val, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer, random_state=random_state)
    
    elif set_noise_type == 'white_no_error':
        noise_init = 1
        noise_bounds = (1e-4, 20)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer, random_state=random_state)
    
    elif set_noise_type == 'white_mean_error':
        if E_train is None:
            raise ValueError("E_train must be provided for 'white_mean_error'.")
        E_var = E_train**2
        ##### scaling the mean by delta 0.3 for optimising range. Idea  - Keep tight bounds for noise
        noise_init = np.mean(E_var)
        noise_bounds = (np.mean(E_var)*0.7, np.mean(E_var)*1.3)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer, random_state=random_state)
    
    elif set_noise_type == 'white_minmax_error':
        if E_train is None:
            raise ValueError("E_train must be provided for 'white_minmax_error'.")
        E_var = E_train**2
        ##### bounds for noise level decided by min and max of the error
        min_e = np.min(E_var)
        max_e = np.max(E_var)
        noise_init = np.mean(E_var)
        noise_bounds = (min_e, max_e)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer,random_state=random_state)
    
    else:
        raise ValueError("Invalid noise type.")

In [ ]:
def cross_validate_gpr(kernel, modeltypes, X_trainval, D_trainval, E_trainval, folds=folds):
    """
    Runs cross-validation for a given kernel and list of model types (noise types).
    
    Parameters:
    -----------
    kernel : sklearn kernel object
        The base kernel (e.g., RBF(), RationalQuadratic(), Matern()).
    modeltypes : list of str
        List of noise model types (e.g., ['white_no_error', 'white_mean_error', ...]).
    X_trainval, D_trainval, E_trainval : ndarray
        Training/validation data and associated error.
    folds : int
        Number of folds (default 10).
    
    Returns:
    --------
    cv_results : dict
        Dictionary with metrics for each model type.
    """

    cv_results = {}
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)

    for modeltype in modeltypes:
        print(f"\nRunning CV for model type: {modeltype}")

        r2, rmse, mae, adjusted_r2, fom, pearsoncoeff = [], [], [], [], [], []

        for train_index, val_index in tqdm(list(kf.split(X_trainval)), total=folds, desc=f"CV {modeltype}"):
            X_train, X_val = X_trainval[train_index], X_trainval[val_index]
            D_train, D_val = D_trainval[train_index], D_trainval[val_index]
            E_train, _ = E_trainval[train_index], E_trainval[val_index]

            gpr = build_gpr(kernel,modeltype, E_train)
            gpr.fit(X_train, D_train)

            D_pred, D_std = gpr.predict(X_val, return_std=True)


            rmse.append(np.sqrt(mean_squared_error(D_val, D_pred)))
            mae.append(mean_absolute_error(D_val, D_pred))
            r2.append(r2_score(D_val, D_pred))
            adjusted_r2.append(1 - (1 - r2[-1]) * (len(D_val) - 1) / (len(D_val) - X_val.shape[1] - 1))
            fom.append(np.mean(np.abs(D_val - D_pred) / D_std))
            corr, _ = pearsonr(D_val.ravel(), D_pred.ravel())
            pearsoncoeff.append(corr)
        cv_results[modeltype] = {'r2': r2,'rmse':rmse,'mae':mae,'adjusted_r2':adjusted_r2, 'fom':fom,'pearson': pearsoncoeff}

    return cv_results

In [ ]:
bounds_lmult = ((1e-4, 10), (1e-4, 10.0), (1e-4, 10.0), (1e-4, 10.0))  # Fix l1 bounds
guess_lmult = (1,1,1,1)
bounds_lsingle = (1e-4, 10.0)
ExpSinelbounds = (1e-1, 10.0)
guess_lsingle = 1.0
guess_signal_var = 1.0
bounds_signal_var = (1e-2, 20)
nu = 1.75
alpha = 1
periodicity = 2
periodictybounds = (1, 20)
gamma = 1
Matkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * Matern(length_scale=guess_lmult, length_scale_bounds=bounds_lmult, nu=nu)
Radkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RationalQuadratic(length_scale=guess_lsingle, alpha=alpha, length_scale_bounds=bounds_lsingle)
ExpSinekernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * ExpSineSquared(length_scale=guess_lsingle, periodicity=periodicity, length_scale_bounds=ExpSinelbounds,periodicity_bounds=periodictybounds)
Laplacekernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * PairwiseKernel(gamma=gamma, metric='laplacian')
RBFkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=guess_lmult, length_scale_bounds=bounds_lmult)
kernels = {
    "Matern": Matkernel,
    "RationalQuadratic": Radkernel,
    "ExpSineSquared": ExpSinekernel,
    "Laplace": Laplacekernel,
    "RBF": RBFkernel
}
modeltypes = ['white_no_error', 'white_mean_error','white_minmax_error','fixed_alpha']

cv_results_all_kernels = {}
for kernel_name, kernel_obj in kernels.items():
    print(f"\n\nRunning CV for kernel: {kernel_name}")
    cv_results = cross_validate_gpr(kernel_obj, modeltypes, X_trainval, D_trainval, E_trainval, folds=folds)
    # Store results in the final dictionary under kernel name
    cv_results_all_kernels[kernel_name] = cv_results



Running CV for kernel: Matern

Running CV for model type: white_no_error


CV white_no_error:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
CV white_no_error:  20%|██        | 2/10 [09:37<37:52, 284.07s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
CV white_no_error: 100%|██████████| 10/10 [48:21<00:00, 290.14s/it]



Running CV for model type: white_mean_error


CV white_mean_error:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.5143804386540594. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_mean_error:  10%|█         | 1/10 [04:14<38:14, 254.90s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.5267006904199005. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_mean_error:  20%|██        | 2/10 [08:23<33:30, 251.27s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level 


Running CV for model type: white_minmax_error


...

CV white_mean_error:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.5143804386540594. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_mean_error:  10%|█         | 1/10 [01:05<09:47, 65.26s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.5267006904199005. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_mean_error:  20%|██        | 2/10 [02:14<09:03, 67.88s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is


Running CV for model type: white_minmax_error


CV white_minmax_error:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.012608095136544591. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_minmax_error:  10%|█         | 1/10 [01:22<12:21, 82.42s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.012608095136544591. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_minmax_error:  20%|██        | 2/10 [02:48<11:15, 84.40s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__nois


Running CV for model type: fixed_alpha


CV fixed_alpha: 100%|██████████| 10/10 [07:39<00:00, 45.92s/it]


In [8]:
cv_results_all_kernels
joblib.dump(cv_results_all_kernels, 'cv_results_all_kernels.pkl')

['cv_results_all_kernels.pkl']

In [9]:
print(cv_results_all_kernels)

{'Matern': {'white_no_error': {'r2': [0.9886976608532412, 0.9760740590775828, 0.991233208518017, 0.9170505401249724, 0.9944755287601859, 0.9889131540249749, 0.976174577542878, 0.9636008595831126, 0.9802819465348045, 0.9835647857400173], 'rmse': [0.03733507755532921, 0.05678286034521193, 0.03372613940337225, 0.10318332867242225, 0.028248535792664724, 0.0382894742763294, 0.04800067207213081, 0.06844070834953277, 0.05003631926938455, 0.04599602616623826], 'mae': [0.02587154824549528, 0.027992088947022077, 0.023368207064117186, 0.042313073436945105, 0.022941167112427482, 0.02880565050766248, 0.03172696512872745, 0.035727285235143985, 0.028935265956615233, 0.028829977497744814], 'adjusted_r2': [0.9881657860698643, 0.9749481324459397, 0.9908206536247472, 0.9131470361308535, 0.9942155536430182, 0.9883914200967384, 0.9750533811919546, 0.961887958857612, 0.9793540381364424, 0.9827913638924887], 'fom': [0.5130438880456283, 0.5325448397403203, 0.4677785368967309, 0.9581675612632756, 0.47243406681

In [10]:
#### Converting to csv
records = []
for modeltype, metrics in cv_results.items():
    for i in range(len(metrics['r2'])):  # number of folds
        record = {'modeltype': modeltype}
        for metric_name, values in metrics.items():
            record[metric_name] = values[i]
        records.append(record)

df = pd.DataFrame(records)
df.to_csv('cv_results.csv', index=False)